In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import re
import os
import csv
import pyperclip

/Users/enriccogemha/Developer/courses-at-Insper/social_networks/aps03/web_scrapper/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# set up the PATHs
URL_WEBSITE = 'https://limitlesstcg.com/tournaments?time=12months&type=all&format=standard&region=all&show=100'
URL_WEBSITE_TEST = 'https://limitlesstcg.com/tournaments?time=1months&type=all&format=standard&region=all&show=100'

In [3]:
# I need to use selenium to scrape the data from the website below
# https://limitlesstcg.com/tournaments?time=12months&type=all&format=standard&region=all&show=100
# And extract the decklists from the top 8 players of each tournament and save them in a csv file
# The csv file should have the following columns:
# Tournament Name, Date, Player Name, Deck Name, Card1, Card2, ..., Card60
# The csv file should have a row for each decklist
# begin now


# Initialize the WebDriver (in this case, for Chrome)
driver = webdriver.Chrome()

# Go to the limitlesstcg website
driver.get(URL_WEBSITE_TEST)

# Get all the urls for the tournaments

# all elements have a tag <a> and the href="/tournament/xxxxx" where xxxxx is the tournament id
# I can use the xpath to find all the elements with the tag <a> and then extract the href attribute
# wait for the page to load
time.sleep(1)

elements = driver.find_elements(By.XPATH, '/html/body/main/div/table/tbody//td[3]/a')
urls = []
for element in elements:
    urls.append(element.get_attribute('href'))

print(len(urls))
print(urls[0])

3
https://limitlesstcg.com/tournaments/385


In [8]:
deck_urls = []

for url in urls:
    driver.get(url)
    time.sleep(1)
    # get the link for the decks of the top 8 players
    elements = driver.find_elements(By.XPATH, '/html/body/main/div/table/tbody//td[4]/a')
    i = 0
    for element in elements:
        deck_url = element.get_attribute('href') + '/cards'
        if len(element.get_attribute('href').split('?')) == 2:
            deck_url = (element.get_attribute('href').split('?')[0] + '/cards?' + element.get_attribute('href').split('?')[1])
        deck_urls.append(deck_url)
        i += 1
        if i == 8:
            break

print(len(deck_urls))
print(deck_urls)

24
['https://limitlesstcg.com/decks/264/cards', 'https://limitlesstcg.com/decks/240/cards?variant=1', 'https://limitlesstcg.com/decks/264/cards', 'https://limitlesstcg.com/decks/264/cards?variant=1', 'https://limitlesstcg.com/decks/212/cards?variant=38', 'https://limitlesstcg.com/decks/264/cards?variant=1', 'https://limitlesstcg.com/decks/255/cards', 'https://limitlesstcg.com/decks/264/cards', 'https://limitlesstcg.com/decks/224/cards?variant=2', 'https://limitlesstcg.com/decks/255/cards', 'https://limitlesstcg.com/decks/264/cards?variant=3', 'https://limitlesstcg.com/decks/247/cards', 'https://limitlesstcg.com/decks/255/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/241/cards', 'https://limitlesstcg.com/decks/257/cards?variant=2', 'https://limitlesstcg.com/decks/211/cards?variant=1', 'https://limitlesstcg.com/decks/247/cards', 'https://limitlesstcg.com/decks/240

In [9]:
i = 0
for deck_url in deck_urls:
    driver.get(deck_url)
    time.sleep(1)
    # get the button by class='export'
    button = driver.find_element(By.CLASS_NAME, 'export')
    button.click()
    copied_text = pyperclip.paste()
    # save the text in a file
    with open(f'output/decklist{i}.txt', 'w') as f:
        f.write(copied_text)
    i += 1

In [10]:
driver.quit()